## Start

In [ ]:
import clickhouse_connect
import datetime
import os
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv

import sys
sys.path.append('/home/boris/Documents/Work/analytics/Clickhouse')
from clickhouse_client import ClickHouse_client
ch = ClickHouse_client()
pd.set_option('display.max_rows', 1000)

___
### Tags: #Mobile_Report

### Links:  

[[installation_point_st_partner]]

[[intercoms_st_partner]]

[[entries_installation_points_dir_partner]]

[[companies_st_partne]]


### Table

In [ ]:
query_text = """--sql
CREATE TABLE db1.units_on_sk_platform_rep_mobile_total
(
    `report_date` Date,
    `partner_uuid` String,
    `city` String,
    `units_on_platform` UInt64,
    `units_stricted monetization` UInt64,
    `units_free_monetization` UInt64,
    `units_free_monetization_pro` UInt64,
    `units_free_monetization_start` UInt64
)
ENGINE = MergeTree()
ORDER BY partner_uuid
"""

ch.query_run(query_text)

In [ ]:
query_text = """--sql
    CREATE MATERIALIZED VIEW db1.units_on_sk_platform_rep_mobile_total_mv
    REFRESH EVERY 1 DAY OFFSET 6 HOUR 8 MINUTE TO db1.units_on_sk_platform_rep_mobile_total AS
    SELECT
        inst_p_st.report_date AS report_date,
        inst_p_st.`partner_uuid` AS partner_uuid,
        entr_p_dir.city AS city,
        count(distinct if(intercom_uuid is not NULL, `address_uuid`,null)) as `units_on_platform`,
        count(distinct if(monetization = 0 and intercom_uuid is not NULL, `address_uuid`,null)) as `units_stricted monetization`,
        count(distinct if(monetization = 1 and intercom_uuid is not NULL, `address_uuid`,null)) as `units_free_monetization`,
        count(distinct if(monetization = 1 and intercom_uuid is not NULL  and pro_subs = 1, `address_uuid`,null)) as `units_free_monetization_pro`,
        count(distinct if(monetization = 1 
                        and intercom_uuid is not NULL 
                        and pro_subs = 1
                        and (enterprise_subs = 0 or enterprise_subs is null) 
                        and (enterprise_not_paid = 0 or enterprise_not_paid is null)  
                        and (enterprise_test = 0 or enterprise_test is null)
                        , `address_uuid`,null)) as `units_free_monetization_start`
    FROM db1.`installation_point_st_partner_ch` AS inst_p_st
    ANY JOIN db1.`intercoms_st_partner_ch` AS int_st
                    ON int_st.installation_point_id = inst_p_st.installation_point_id
                    AND int_st.report_date = inst_p_st.report_date
    LEFT ANY JOIN db1.`entries_installation_points_dir_partner_ch` AS entr_p_dir 
        ON inst_p_st.installation_point_id = entr_p_dir.installation_point_id
    LEFT ANY JOIN db1.`companies_st_partner_ch` AS comp_st
                    ON comp_st.`partner_uuid` = int_st.`partner_uuid`
                    AND comp_st.`report_date` = int_st.`report_date`
    GROUP BY  
        report_date,
        partner_uuid,
        city
    """
ch.query_run(query_text)

In [ ]:
mobile_report_rep_mobile_full

___
## Tools
___
### query


In [12]:
query_text = """--sql
    SELECT
        *
    FROM db1.units_on_sk_platform_rep_mobile_total
    ORDER BY report_date DESC
    limit 100
    """

ch.query_run(query_text)

report_date,partner_uuid,city,units_on_platform,units_stricted monetization,units_free_monetization,units_free_monetization_pro,units_free_monetization_start
date,str,str,i64,i64,i64,i64,i64
2025-08-04,"""00be0483-e667-4d7e-a462-64f1eb…","""Полевской""",22,22,0,0,0
2025-08-04,"""010cd1e6-7f7b-4613-9209-96b28d…","""рабочий посёлок Вычегодский""",47,0,47,47,47
2025-08-04,"""010cd1e6-7f7b-4613-9209-96b28d…","""Коряжма""",140,0,140,140,140
2025-08-04,"""00ec6d8e-a799-409c-958b-9472a1…","""Казань""",12,8,4,0,0
2025-08-04,"""00be0483-e667-4d7e-a462-64f1eb…","""Екатеринбург""",11,11,0,0,0
…,…,…,…,…,…,…,…
2025-08-04,"""13463781-c891-4451-9498-746158…","""Новый Уренгой""",10,5,5,0,0
2025-08-04,"""10e08a08-863c-4932-b351-31ec59…","""Москва""",2,0,2,0,0
2025-08-04,"""11cae268-bf50-4705-a05e-138bce…","""Аксай""",1,0,1,0,0


### delete a part


In [ ]:
query_text = """--sql
    ALTER TABLE db1.units_on_sk_platform_rep_mobile_total DELETE WHERE report_date = '2025-07-17'
    """

ch.query_run(query_text)

### drop mv

In [ ]:
query_text = """--sql
    DROP TABLE db1.units_on_sk_platform_rep_mobile_total_mv
    """

ch.query_run(query_text)

### drop ch

In [ ]:
query_text = """--sql
    DROP TABLE db1.units_on_sk_platform_rep_mobile_total
    """

ch.query_run(query_text)

### refresh mv

In [ ]:
query_text = """
SYSTEM REFRESH VIEW db1.units_on_sk_platform_rep_mobile_total_mv
"""

ch.query_run(query_text)